In [14]:
import sys
sys.path.append('C:/Users/dsill/Desktop/stock-prediction')
from davidplayground.datasets import *
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
sns.set()

In [6]:
grt = CoinDataset(name="GRT")
grtdf = grt.get(start="2020-12-31", end="2021-08-28", granularity=300)

In [31]:
"""
Data Prep: 
* For each day, Z-normalize the prices.
    * For each hour within each day, save the last hour's prices as a window.
    * If the price will rise by the next hour, classify as +1 and -1 if it decreases.
"""
prices = ['open', 'high', 'low', 'close']
dataset = []
classes = []
for d in range(len(grtdf) // (12*24)):
    daystart = d * (12*24)
    daydata = grtdf.loc[daystart:daystart+12*24,prices]
    hourdata = np.zeros((23,12,4))
    hourclas = np.zeros((23,1))
    for h in range(0, 23):
        row = daydata.iloc[h*12:(h+1)*12,:].values
        nextrow = daydata.iloc[(h+1)*12:min((h+2)*12, len(daydata)),:].values
        hourdata[h] = row
        hourclas[h] = 1 if nextrow[-1,3] > row[-1,3] else -1
    hourdata_scaled = MinMaxScaler().fit_transform(hourdata.flatten().reshape(-1,1)).reshape(hourdata.shape)
    dataset.extend(hourdata_scaled)
    classes.extend(hourclas)
dataset = np.array(dataset)
classes = np.array(classes)
print(len(dataset), len(classes))

5543 5543


In [33]:
data = []
for i in range(len(dataset)):
    hexhash = hex(hash(str(dataset[i])))
    hexhash = hexhash[hexhash.rfind('x')+1:]
    img = Image.fromarray(dataset[i].T*1.0)
    imgpath = f'dataset/images/{hexhash}.tiff'
    img.save(imgpath)
    data.append({'name':imgpath, 'class':classes[i][0]})
datadf = pd.DataFrame(data)
datadf.to_csv('dataset/dataset.csv', index=False)